# This notebook run code(s) on multiple animal/session/ for RAT Behavioral experiment

### This notebook is at the core of the pipeline of data processing. Do not play with it lightly inside the master folder (load_preprocess_rat)

#### 1. Only modifiy if you are sure of what you are doing and that you are solving a bug
#### 2. If you do modify you MUST commit this modification using bitbucket
#### 3. If you want to play whis notebook (to understand it better) copy it on a toy folder distinct from the master folder
#### 4. If you want to modify this code (fix bug, improve, add attributes ...) it is recommanded  to first duplicate in a draft folder. Try to keep track of your change.
#### 5. When you are ready to commit : # clear all output, clean everything between hashtag 



## 1. Load packages and define functions

In [14]:
#modules to find path of all sessions
import glob
import os
import numpy as np
from IPython.display import clear_output, display, HTML
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
#run other notebooks
## The lines below allow to run required notebook from the master folder
if "__file__" not in dir():
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommunNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    os.chdir(CommunNoteBookesPath)   
    %run plotRat_documentation_1_GeneralBehavior_orig.ipynb
    %run plotRat_documentation_2_Spiking.ipynb
    %run loadRat_documentation.ipynb
    %run loadRawSpike_documentation.ipynb



# PARAMETERS (used if the pickles don't exist)
param={
    "goalTime":7,#needed for pavel data only
    "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
    "maxTrialDuration":20,
    "interTrialDuration":10,#None pavel
    "endTrial_frontPos":30,
    "endTrial_backPos":55, 
    "endTrial_minTimeSec":4,
    "cameraSamplingRate":25, #needed for new setup    

    "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
    "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
     "nbJumpMax":100,#200 pavel
    "binSize":0.25,
    #parameters used to preprocess (will override the default parameters)
}  

if platform.system()=='Linux':
    root="/data"
else:
    root="/Users/davidrobbe/Documents/Data/"
    

def has_tag(root, animal, session, tagList):
    """ Test if the session has at least one of the tag in tagList
    tag = empty file with a specific name, in a session folder
    """
    fullPath = os.path.join(root, animal, "Experiments", session)
    fileList = os.listdir(fullPath)
    for tag in tagList:
        if tag in fileList:
            return True
    return False    
    
    
# below you can manual modify the root folder. Before commit or push remove anything between these line and leave 1 line blank

##############################

############################## 
    
print("path to data is %s"%root)
print("")
print("importing done")

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 41648: ordinal not in range(128)

path to data is /Users/davidrobbe/Documents/Data/

importing done


## 2. Define animalList, the function(s) you want to run in batch and redo(s)

In [6]:
#Example of animal list DO not changes the example below!!!

# all Rats
animalList=[os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
animalList = ['Rat101', 'Rat102', 'Rat103', 'Rat104']
animalList = ['Rat079']

#you can change animal list in between the hashtag 
###################

###################

#print("Available list of animals: %s"%animalList)

# this is where you define a list of functions you want to run on your data
plotFunctionList=[plot_session_behavior]
###################

###################

redo = False #redo basic preprocess and generate Data. Otherwise load from pickle 
###################

###################
override = False #regenerate single session png figure. Otherwise use existing
###################

###################
replaceHtml=False #regenerate general HTML per animal. Otherwise use existing
###################

###################
tagList=[] ## some session are tagged to be specifically analyzed
###################

###################

NameError: name 'plot_session_behavior' is not defined

## 3 - Loops through animals and session and functions

In [14]:
#===================================================================================
''' Option A: Plot functions "one plot per session"  - BEHAVIOR'''
#
# one session = one folder = one experiment
#
# A plot is done in a "plot function", either defined here or in another .py file (ex: plotData.py). 
#
# You can do several plots at once (by giving a list of "plot function").
#
# For every session, the plots are saved in *RatXXX/Experiments/RatXXX_20.../nameOfPlot.png*  
#
# An html page is created or updated: *RatXXX/Analysis/all_nameOfPlot.html*
#===================================================================================

#-------------list of plot functions to do on each session--------------------
#those functions should take as arguments: data (+ optionnal keywords arguments)
#the functions can return False (then no plot will be saved)

print("List of animal to loop through: %s"%animalList)
plotFunctionList=[plot_session_behavior]
errorSession=[]


#---------------------------loop on animal-------------------------------------

for animal in animalList:
    print("Animal %s"%animal)
    print("**********")
    
    #----Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+"/"+animal+"/Experiments/Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)
    
    
    #----If delete the general HTML file
    
    if replaceHtml:
        for plotFunction in plotFunctionList:
            name = str(plotFunction.__name__)

            #html for the animal
            generalName = "all_" + name
            AnalysisFolder = os.path.join(root, animal, "Analysis")
            generalPath = os.path.join(AnalysisFolder, generalName+".html")
            if os.path.exists(generalPath):
                os.remove(generalPath)
    
    #----loop through sessions
    for index,session in enumerate(sessionList):
        #clear_output()
        print("Session %s/%s: %s"%(index+1,nbSession,session))
        
        #----------------------check for tag file-------------------------
        #if tag list is not emtpy
        if tagList:
            #check if the session has one of the tag
            if not has_tag(root, animal, session, tagList):
                continue

        #----load data for this session (add redoPreprocess=True to overwrite preprocess)
        try:
            sessionData=Data(root,animal,session,param=param,redoPreprocess=redo)
        except Exception as e:
            display(HTML("<b> An error happened, skip session. Error message: </b>"))
            display(HTML("<b>"+e.__class__.__name__+": "+str(e)+"</b>"))
            errorSession.append(session)
            continue            
        #----skip if training data or no break time
        if (not sessionData.hasBehavior):
            print("No Behavior, skipped")
            print("--------")
            continue  
        #---skip if file "NotToAnalyze"
        try:
            if "NotToAnalyze" in sessionData.emptyAnalysisFiles:
                print("Not to analyze, skipped")
                print("--------")
                continue 
        except:
            continue
        #----plot the data (add name="myNewName" to change the name of the html, add override=True to redo the plots)
        sessionData.plot_session_png_html(plotFunctionList,override=override)
        clear_output()
#---------------------------------------------------------------------------------------       
print("Done")
if errorSession:
    display(HTML("<b> An error happened for the following session(s):"))
    print("\n".join(errorSession))
#=======================================================================================

Done


In [ ]:

#===================================================================================
''' Option B: Plot functions "one plot per cluster"  - SPIKES'''
#
# A plot is done in a "plot function", either defined here or in another .py file (ex: plotData.py).   
#
# You can do several plots at once (by giving a list of "plot function")
#
# For every session, the plots are saved in *RatXXX/Experiments/RatXXX_20.../plots/shankX_clusterX_nameOfPlot.png*  
#
# Two html pages are created or updated: 
#  - *RatXXX/Experiments/RatXXX_20.../nameOfPlot.html*: all plots for session ("session html")
#  - *RatXXX/Analysis/all_nameOfPlot.html*: links to every "session html" for this rat
#
#===================================================================================

#-------------list of plot functions to do on each session--------------------
#those functions should take as arguments: data,shank,cluster,group (+ optionnal keywords arguments)
#the functions can return False (then no plot will be saved)
animalList=["Rat025"]
plotFunctionList=[plot_raster_correlogram,plot_waveforms]

#------------------------------loop on animal----------------------------------------------  
for animal in animalList:
    print("animal %s"%animal)
    print("**********")

    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+"/"+animal+"/Experiments/Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)

    errorSession=[]
    
    #loop through sessions
    for index,session in enumerate(sessionList):
        print("Session %s/%s: %s"%(index+1,nbSession,session))
        #load data for this session (add redoPreprocess=True to overwrite preprocess)
        #----------------------check for tag file-------------------------
        isTagIn = True
        if len(tagList)>0:
            isTagIn = False
            for tag in tagList:
                    if os.path.exists(root+"/"+animal+'/Experiments'+'/'+session+'/Analysis/'+tag):
                        isTagIn = True
            if not isTagIn: 
                print("Tag not found, session skipped")
                print("--------")
                continue

        #----load data for this session (add redoPreprocess=True to overwrite preprocess)
        try:
            sessionData=Data(root,animal,session,param=param,redoPreprocess=redo)
        except Exception as e:
            display(HTML("<b> An error happened, skip session. Error message: </b>"))
            display(HTML("<b>"+e.__class__.__name__+": "+str(e)+"</b>"))
            errorSession.append(session)
            print("--------")
            continue
            
        #skip if training data or no break time
        if (not sessionData.hasBehavior):
            print("No behavior, skipped")
            print("--------")
            continue
            
        #skip if file "NotToAnalyze"
        if "NotToAnalyze" in sessionData.emptyAnalysisFiles:
            print("Not to analyze, skipped")
            print("--------")
            continue        
            
        if sessionData.hasSpike:
            #plot the data (add name="myNewName" to change the name of the html, add override=True to redo the plots)
            # groupList can be "Good", ["Good","MUA"],... No groupList -> do every group
            sessionData.plot_all_clusters_png_html(plotFunctionList,override=override,groupList="Good")
        print("--------")
        #---------------------------------------------------------------------------------------            
        print("Done")
        if errorSession:
            display(HTML("<b> An error happened for the following session(s):"))
            print("\n".join(errorSession))
        #=======================================================================================